# Vector AutoRegression model

Attempting to predict wind speed, direction and temp

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
ZEPHYR_DB_PATH = "../zephyr-model.db"
TARGET_STATION = "Coronet Summit"
PREDICTOR_STATION_MAX_DISTANCE_KM = 20 # Use stations within this radius to predict

In [ ]:
def get_query(sql: str, params: list = []) -> pd.DataFrame:
    """
    Execute `sql` with `params` and return the results as a data frame
    """
    conn = sqlite3.connect(ZEPHYR_DB_PATH)
    cursor = conn.cursor().execute(sql, params)
    res = cursor.fetchall()
    colnames = [x[0] for x in cursor.description]
    datatypes  = [x[1] for x in cursor.description]
    data = [dict(zip(colnames, row)) for row in res]
    schema = {col: dt for col, dt in zip(colnames, datatypes)} # Not working for some reason
    df = pd.DataFrame(data)
    return df


Which stations do we want to use to predict?

In [ ]:
predictor_station_sql = f"""
select
    to_station.name,
    to_station.id,
    dist.km_between as station_distance
from
    station_distances as dist
    join stations as from_station on from_station.id = dist.id_from
    join stations as to_station on to_station.id = dist.id_to
where
    from_station.name = '{TARGET_STATION}'
    and dist.km_between <= {PREDICTOR_STATION_MAX_DISTANCE_KM}
    order by dist.km_between
"""
predictor_stations = get_query(predictor_station_sql)

Get observations from our predictor stations.

In [ ]:
predictor_sql = f"""
select id, name, timestamp, temperature, wind_average, wind_gust, wind_bearing
from observations
where id in ({','.join("?" * len(predictor_stations["id"]))})
"""

predictor_data = get_query(predictor_sql, predictor_stations["id"])

Get our ground truth data for the station of interest.

In [ ]:
target_sql = f"""
select id, name, timestamp, temperature, wind_average, wind_gust, wind_bearing
from observations
where name = '{TARGET_STATION}'
"""

target_data = get_query(target_sql)